In [8]:
#验证模型生成分子效果
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image

from transvae.trans_models import TransVAE
from transvae.rnn_models import RNN, RNNAttn
from transvae.tvae_util import *
%load_ext autoreload
%autoreload 2

import IPython.display as Disp
np.set_printoptions(suppress=True)
# Load model and set of SMILES
vae = TransVAE(load_fn='checkpoints/best_transvae_Inhibitor1368_data_2_model.ckpt')
#vae = RNNAttn(load_fn='checkpoints/best_rnnattn_Inhibitor1368_data_2_model.ckpt')
#vae = RNN(load_fn='checkpoints/best_rnn_Inhibitor1368_data_2_model.ckpt')
smiles_data = pd.read_csv('Inhibitor1368_data_2/val_mols_0.9_new.txt', sep='\t', header=None)
smiles = smiles_data.iloc[:, 0].to_numpy()
#smiles_data = pd.read_csv('Inhibitor1368_data/val_mols.txt').to_numpy()
#smiles = smiles_data[:,0]
smiles = smiles.reshape(-1, 1)
print(smiles.shape)
props_data = pd.read_csv('Inhibitor1368_data_2/val_props_0.9_new.txt', sep='\t', header=None, usecols=[0,1])
    # 转换为NumPy数组
props = props_data.to_numpy()
#print(props.shape)
#print(props)
vae.params['BATCH_SIZE'] = 1
vae.params['BATCH_CHUNKS'] = 1
#reconstructed_smiles = vae.reconstruct_log1(data=smiles,props=props,target_ie=0.955, target_concentration=1, method='greedy', num_reconstructions=1000,noise_type='gaussian', base_noise_std=-0.1, noise_factor=1.1, log=True, return_mems=True, return_str=True)
reconstructed_smiles = vae.reconstruct_rand(data=smiles,props=props,target_ie=0.955, target_concentration=1, method='greedy', num_reconstructions=1000,noise_type='uniform',base_noise_std=0, max_noise_std=1, log=True, return_mems=True, return_str=True)
import pandas as pd
for item in reconstructed_smiles:
    print(type(item))
# 访问元组中的第一个元素（即列表）
list_data = reconstructed_smiles[0]
Filename="Inhibitor1368_data_2/TransVAE_rand_uniform_0.955_std_0.1"
# 假设 reconstructed_smiles 是一个包含列表的元组
list_data = reconstructed_smiles[0]
#print(list_data)
results_df = pd.DataFrame(list_data, columns=['SMILES', 'Predicted_IE', 'Concentration_M'])
results_df.to_csv(Filename+'_results.csv', index=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
self.src_len: 126
(1, 1)


C:\Users\DELL\Desktop\train_Inhibitor1368_data_2_new\transvae\trans_models.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


Loaded model from best_rf_model.joblib
1
<class 'list'>
<class 'numpy.ndarray'>


In [44]:
for item in reconstructed_smiles:
    print(type(item))
# 访问元组中的第一个元素（即列表）
list_data = reconstructed_smiles[0]

# 假设 reconstructed_smiles 是一个包含列表的元组
list_data = reconstructed_smiles[0]
#print(list_data)
# 打开一个文件用于写入
Filename="Inhibitor1368_data_2/TransVAE_gaussian_0.982_std_-0.1"
with open(Filename+'_gen_mols.txt', 'w') as file:
    # 遍历列表中的每个元素
    for item in list_data:
        # 写入元素到文件，后面跟一个制表符
        file.write(f"{item}\n")
    # 在文件末尾写入一个换行符，以结束文件内容
    file.write("\n")

# 文件自动保存并关闭


<class 'list'>
<class 'numpy.ndarray'>


In [45]:
import re
import math
import copy
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from rdkit import rdBase
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect as Morgan

rdBase.DisableLog('rdApp.*')

def valid(smiles):
    "Returns valid SMILES (RDKit sanitizable) from a set of\
    SMILES strings"
    valid_smiles = []
    for smi in smiles:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            pass
        else:
            try:
                Chem.SanitizeMol(mol)
                valid_smiles.append(smi)
            except ValueError:
                pass
    return valid_smiles


# 读取SMILES分子式数据
with open(Filename+'_gen_mols.txt', 'r') as f:
    smiles_data = f.readlines()

# 移除可能的换行符，并获取有效的SMILES
smiles_data = [smi.strip() for smi in smiles_data]
valid_smiles_data = valid(smiles_data)
print(valid_smiles_data)
# 将有效的SMILES分子式保存到新的文档中
with open(Filename+'_gen_mols_valid_mols.txt', 'w') as f:
    for smi in valid_smiles_data:
        f.write(smi + '\n')

print(f"处理完成，有效的SMILES分子式已保存到 'valid_smiles.txt'。")

['C[N+](C)(CCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCC

In [46]:
#生成的分子评估
def novel(smiles, train_smiles):
    "Returns novel SMILES strings that do not appear\
    in training set"
    set_smiles = set(smiles)
    set_train = set(train_smiles)
    novel_smiles = list(set_smiles - set_train)
    return novel_smiles

def unique(smiles):
    "Returns unique SMILES strings from set"
    unique_smiles = set(smiles)
    return list(unique_smiles)

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs


def get_fingerprints(smiles_list):
    fingerprints = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol:  # 确保mol不是None
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
            fingerprints.append(fp)
    return fingerprints


"""
def tanimoto_similarity(fp_list1, fp_list2):
    "Calculates Tanimoto similarity between two sets of fingerprint bit vectors"
    similarities = []
    for fp1 in fp_list1:
        for fp2 in fp_list2:
            sim = DataStructs.FingerprintSimilarity(fp1, fp2)
            similarities.append(sim)
    return similarities
    """
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

def tanimoto_similarity(smiles_list1, smiles_list2):
    results = []
    # Generate fingerprints, checking for valid molecule conversion
    fps1 = []
    for smiles in smiles_list1:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.RDKFingerprint(mol)
            fps1.append((smiles, fp))
        else:
            print(f"Warning: Failed to generate molecule from SMILES {smiles}")
    
    fps2 = []
    for smiles in smiles_list2:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.RDKFingerprint(mol)
            fps2.append((smiles, fp))
        else:
            print(f"Warning: Failed to generate molecule from SMILES {smiles}")
    
    # Calculate Tanimoto similarity
    for smiles1, fp1 in fps1:
        max_similarity = 0
        best_match_smiles = None
        for smiles2, fp2 in fps2:
            sim = DataStructs.FingerprintSimilarity(fp1, fp2)
            if sim > max_similarity:
                max_similarity = sim
                best_match_smiles = smiles2
        if best_match_smiles is not None:
            results.append((smiles1, best_match_smiles, max_similarity))
        else:
            print(f"No valid matches found for {smiles1}")

    return results

# 读取SMILES分子式数据

# 读取SMILES分子式数据
with open(Filename+'_gen_mols_valid_mols.txt', 'r') as f:
    gen_valid_smiles_data = [line.strip() for line in f.readlines()]

with open('Inhibitor1368_data_2/val_mols_0.9_new.txt', 'r') as f:
    raw_val_mols_data = [line.strip() for line in f.readlines()]
    
with open('Inhibitor1368_data_2/all_SMILES_in.txt', 'r') as f:
    all_mols_data = [line.strip() for line in f.readlines()]

novel_smile=novel(gen_valid_smiles_data, all_mols_data)
print("novel_smile:",novel_smile)
unique_value=unique(gen_valid_smiles_data)
print("unique_value:",unique_value)
"""
fingerprints_gen=get_fingerprints(gen_valid_smiles_data)
print("fingerprints_gen:",fingerprints_gen)

fingerprints_raw=get_fingerprints(raw_val_mols_data)
print("fingerprints_gen:",fingerprints_raw)

best_pair, max_similarity=tanimoto_similarity(fingerprints_gen, fingerprints_raw)
print("tanimoto_similarity:",max_similarity)
# 计算平均值
print("len(max_similarity)",len(max_similarity))
if len(max_similarity) > 0:
    average_similarity = sum(max_similarity) / len(max_similarity)
    print("Average Tanimoto similarity:", average_similarity)
else:
    print("No similarities to average.")
    """
similarity_results = tanimoto_similarity(novel_smile, raw_val_mols_data)

# Save results to a file
with open(Filename+'_gen_mols_valid_mols_novel__data_tanimoto_similarity.txt', 'w') as file:
    for smiles1, smiles2, sim in similarity_results:
        file.write("\t".join((str(smiles1),str(smiles2),str(sim)))+'\n')
        #file.write(f"{smiles1},{smiles2},{sim:.4f}\n")
        

novel_smile: ['CCCCCCCCCCCCCCCC[N+](C)(C)CCCNC(=O)CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC.[Br-]', '', 'C=[N+]([O-])c1ccc2[nH]cnc2c1', 'Clc1ccc(-n2ccnc2)cc1Cl', 'C[N+](C)(CCCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'CCCCCCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/c2ccc([N+](C)(C)CCCCCCCCCCCCCCCCCCCCCCCCCC)cc2)cc1.[Br-].[Na+]', 'CCCCCCCCCCCC[N+](C)(C)CCCNC(=O)c1ccc(/C=N/c2ccc([N+](C)(C)CCCCCCCCCCCCCCCCC)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCCCCBr)cc3)cc2)cc1.[Br-].[Na+]', 'CCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/c2ccc(-c3ccc(/N=C/c4ccc([N+](C)(C)CCCCCCCCCCCCCC)cc4)c(OC)c3)cc2OC)cc1.[Br-].[Br-]', 'CCCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/C(=S)/N=C/c2ccc([N+](C)(C)CCCCCCCCCCCCCCCCCCCC)cc2)cc1.[Br-].[Br-]', 'C[N+](C)(CCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3ccc([N+](C)(C)CCCCCCCCBr)cc3)cc2)cc1.[Br-].[Br-]', 'CCCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/C(=S)/N=C/c2ccc([N+](C)(C)CCCCCCCCCCCCCCCCCCC)cc2)cc1.[Br-].[Br-]', 'CCCC

In [49]:
from rdkit import Chem
from rdkit.Chem import QED, Descriptors, Lipinski, rdMolDescriptors
import pandas as pd

def compute_properties(smiles):
    if pd.isna(smiles):  # 检查smiles是否为NaN
        return None, None  # 如果是NaN，返回None
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        qed = QED.qed(mol)
        logp = Descriptors.MolLogP(mol)
        molwt = Descriptors.MolWt(mol)
        hba = Lipinski.NumHAcceptors(mol)
        hbd = Lipinski.NumHDonors(mol)
        tpsa = rdMolDescriptors.CalcTPSA(mol)
        num_rot_bonds = Lipinski.NumRotatableBonds(mol)
        n_count = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'N'])
        s_count = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'S'])
        o_count = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'O'])
        p_count = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'P'])
        # 将SMILES字符串作为列表的第一个元素返回
        return [smiles, qed, logp, molwt, hba, hbd, tpsa, num_rot_bonds, n_count, s_count, o_count, p_count]
    else:
        # 如果SMILES无效，返回None，同时保留SMILES字符串以便于识别
        return [smiles] + [None] * 11

# 假设txt文件的每行都是一个SMILES字符串
smiles_path = Filename+'_gen_mols_valid_mols_novel__data_tanimoto_similarity.txt'

# 读取TXT文件中的第一列数据
with open(smiles_path, 'r') as file:
    smiles_list = [line.strip().split('\t')[0] for line in file]

# 计算属性
properties_list = [compute_properties(smiles) for smiles in smiles_list]

# 创建DataFrame
properties_df = pd.DataFrame(properties_list, columns=['SMILES', 'QED', 'LogP', 'MolWt', 'HBA', 'HBD', 'TPSA', 'NumRotBonds', 'N_count', 'S_count', 'O_count', 'P_count'])

print(properties_df)

# 可以将结果保存到CSV文件
properties_df.to_csv(Filename+'_gen_mols_valid_mols_novel__data_properties.csv', index=False)


                                               SMILES       QED      LogP  \
0   CCCCCCCCCCCCCCCC[N+](C)(C)CCCNC(=O)CCCCCCCCCCC...  0.047731  20.23870   
1                        C=[N+]([O-])c1ccc2[nH]cnc2c1  0.297287   1.40530   
2                              Clc1ccc(-n2ccnc2)cc1Cl  0.710723   3.17910   
3   C[N+](C)(CCCCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c...  0.027627   8.92140   
4   CCCCCCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/c2ccc([N+...  0.027161  13.44260   
5   CCCCCCCCCCCC[N+](C)(C)CCCNC(=O)c1ccc(/C=N/c2cc...  0.042571   8.00070   
6   C[N+](C)(CCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3c...  0.026489   7.36100   
7   CCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/c2ccc(-c3ccc(...  0.027115  10.63580   
8   CCCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/C(=S)/N=C/c2...  0.029257  10.00400   
9   C[N+](C)(CCCCCCCCCBr)c1ccc(/C=N/c2ccc(/N=C/c3c...  0.047002   6.19070   
10  CCCCCCCCCCCCC[N+](C)(C)c1ccc(/C=N/C(=S)/N=C/c2...  0.029730   9.61390   
11  CCCCCCCCCCCC[N+](C)(C)CCCNC(=O)C[N+](C)(C)C/N=...  0.116388  -0.34040   

In [50]:
"""读取csv文件，有SMILES，QED，LogP，MolWt，HBA，HBD，TPSA，NumRotBonds几个字段，
要求增加一列Concentration_mM，其中Concentration_mM值可选1,0.1,0.01,0.001，0.005，0.05，5，2，0.5,0.4,0.3，也就是说每行复制为11行，
然后将Concentration_mM值填充到Concentration_mM值列，最后保存到新的csv文件
"""
import pandas as pd

# 读取CSV文件
df = pd.read_csv(Filename+'_gen_mols_valid_mols_novel__data_properties.csv')

# 定义Concentration_mM的可能值
concentration_values = [1, 0.1, 0.01, 0.5, 0.05, 0.001,5, 0.25, 0.005, 2, 0.2,0.4, 0.8,2.5,1.2,0.3,0.75,0.6,1.5,3,0.0075,1.25,0.15,4]
# 重复每行，并为每个重复行分配不同的Concentration_mM值
# 创建一个空的DataFrame来存储最终结果
expanded_df = pd.DataFrame()

for index, row in df.iterrows():
    # 为当前行创建一个临时DataFrame，包含所有Concentration_mM值的重复行
    temp_df = pd.DataFrame([row] * len(concentration_values), columns=df.columns)
    temp_df['Concentration_mM'] = concentration_values
    
    # 将临时DataFrame添加到最终结果DataFrame
    expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)

# 将结果保存到新的CSV文件
expanded_df.to_csv(Filename+'_gen_mols_valid_mols_novel__data_properties.csv', index=False)


In [53]:
"""
使用最优模型对新的CSV文件中的数据进行预测，并将预测结果保存回该CSV文件。这一过程包括读取新数据、预测IE值、添加预测结果到原始DataFrame，然后保存DataFrame。
"""
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

import pandas as pd
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
import joblib
# 自定义RMSE评分函数
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

# 1. 加载数据
df = pd.read_csv('Inhibitor1368_molecular_properties_new_N_O_S_P.csv')

# 2. 数据预处理
# 检查缺失值
if df.isnull().values.any():
    df = df.dropna()  # 删除有缺失值的行

# 3. 设置特征和目标变量
X = df[['QED', 'LogP', 'MolWt', 'HBA', 'HBD', 'TPSA', 'NumRotBonds', 'Concentration_mM','N_count', 'S_count', 'O_count', 'P_count']]
y = df['IE']

# 4. 定义参数网格
param_grid = {
    'n_estimators': [50, 100,150, 200],
    'max_depth': [None, 10,15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3,4]
}

# 5. 建立模型
model = RandomForestRegressor(random_state=42)

# 6. 定义K折交叉验证方法
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 7. 定义评分字典
scoring = {'RMSE': make_scorer(rmse, greater_is_better=False),
           'MAE': 'neg_mean_absolute_error',
           'R2': 'r2'}

# 8. 网格搜索
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring=scoring, refit='RMSE', verbose=3, n_jobs=-1)
grid_search.fit(X, y)

# 9. 打印最优参数及性能指标值
print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", -grid_search.best_score_)
print("Best MAE:", -grid_search.cv_results_['mean_test_MAE'][grid_search.best_index_])
print("Best R2:", grid_search.cv_results_['mean_test_R2'][grid_search.best_index_])

# 10. 将结果存储到DataFrame中
results_df = pd.DataFrame(grid_search.cv_results_)

# 11. 保存结果到CSV文件
results_df.to_csv('RF_IE_grid_search_results_new_O_N_S_P.csv', index=False)




# 假定你已经找到了最优的模型参数
best_params = grid_search.best_params_

# 使用最优参数创建随机森林模型
best_model = RandomForestRegressor(**best_params, random_state=42)

# 使用所有数据重新训练模型（也可以只使用交叉验证中的最佳模型）
best_model.fit(X, y)
# 保存最优模型到文件
model_filename = 'best_rf_model.joblib'
joblib.dump(best_model, model_filename)
print(f"Best model saved to {model_filename}")
# 读取新的CSV文件，这里假设其结构与之前相同，仅没有IE列
new_data = pd.read_csv(Filename+'_gen_mols_valid_mols_novel__data_properties.csv')

# 准备特征数据进行预测
X_new = new_data[['QED', 'LogP', 'MolWt', 'HBA', 'HBD', 'TPSA', 'NumRotBonds', 'Concentration_mM','N_count', 'S_count', 'O_count', 'P_count']]

# 进行预测
predicted_IE = best_model.predict(X_new)

# 将预测结果添加到新数据中的IE列
new_data['IE'] = predicted_IE

# 保存更新后的数据到CSV文件
new_data.to_csv(Filename+'_gen_mols_valid_mols_novel__data_properties_pred_IE.csv', index=False)


Fitting 10 folds for each of 192 candidates, totalling 1920 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best RMSE: 0.09705677340844943
Best MAE: 0.06636209299030339
Best R2: 0.684907697235823
Best model saved to best_rf_model.joblib
